## Prepare the environment and import libraries

In [0]:
%pip install sparkmeasure

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from pyspark.sql import functions as F
from sparkmeasure import StageMetrics
stagemetrics = StageMetrics(spark)
import csv

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-4361612270319640> in <module>
      1 from pyspark.sql import functions as F
      2 from sparkmeasure import StageMetrics
----> 3 stagemetrics = StageMetrics(spark)
      4 import csv

/local_disk0/.ephemeral_nfs/envs/pythonEnv-6884aac6-8412-47b9-9abb-162e52981169/lib/python3.8/site-packages/sparkmeasure/stagemetrics.py in __init__(self, sparksession)
     13         self.sparksession = sparksession
     14         self.sc = self.sparksession.sparkContext
---> 15         self.stagemetrics = self.sc._jvm.ch.cern.sparkmeasure.StageMetrics(self.sparksession._jsparkSession)
     16 
     17     def begin(self):

TypeError: 'JavaPackage' object is not callable

## Read 13 CSV files

In [0]:
Search_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_COVIDSearch.csv")
Education_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_education.csv")
Flight_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_Flight.csv")
Export_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_export.csv")
Immigration_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_immigration.csv")
Lockdown_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_lockdown.csv")
Marriage_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_marriage.csv")
Mask_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_mask.csv")
Medical_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_medicaltreatment.csv")
Social_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_socialdistance.csv")
Travel_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_travel.csv")
Vaccine_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_vaccine.csv")
WFH_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_wfh.csv")

## Select the column in each file we need using the Dataframe

In [0]:
Search_df = Search_df.select(F.col('CountryName'), F.col('Date'), F.col('new_cases'), F.col('Search Index').alias('COVID_search_index'), F.col('new_cases_rate'), F.col('covidsearch_rate').alias('covid_search_rate'))

Education_df = Education_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Education_search_index'), F.col('education_rate'))

Flight_df = Flight_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Flight_search_index'), F.col('flight_rate'))

Export_df = Export_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Export_search_index'), F.col('export_rate'))

Immigration_df = Immigration_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Immigration_search_index'), F.col('immigration_rate'))

Lockdown_df = Lockdown_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Lockdown_search_index'), F.col('lockdown_rate'))

Marriage_df = Marriage_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Marriage_search_index'), F.col('marriage_rate'))

Mask_df = Mask_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Mask_search_index'), F.col('mask_rate'))

Medical_df = Medical_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Medical_search_index'), F.col('medicaltreatment_rate'))

Social_df = Social_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Social_search_index'), F.col('socialdistance_rate'))

Travel_df = Travel_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Travel_search_index'), F.col('travel_rate'))

Vaccine_df = Vaccine_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('Vaccine_search_index'), F.col('vaccine_rate'))

WFH_df = WFH_df.select(F.col('CountryName'), F.col('Date'), F.col('Search Index').alias('WFH_search_index'), F.col('wfh_rate'))


## Join them together

In [0]:
Join_df = Search_df.join(Education_df,['CountryName','Date'], "left").join(Flight_df,['CountryName','Date'], "left").join(Export_df,['CountryName','Date'], "left").join(Immigration_df,['CountryName','Date'], "left").join(Lockdown_df,['CountryName','Date'], "left").join(Marriage_df,['CountryName','Date'], "left").join(Mask_df,['CountryName','Date'], "left").join(Medical_df,['CountryName','Date'], "left").join(Social_df,['CountryName','Date'], "left").join(Travel_df,['CountryName','Date'], "left").join(Vaccine_df,['CountryName','Date'], "left").join(WFH_df,['CountryName','Date'], "left")

print(len(Join_df.columns))
print(Join_df.count())

30
1972


## Convert dataframe to table. And download csv file

In [0]:
#Get All column names and it's types
for name in Join_df.schema.fieldNames():
    Join_df = Join_df.fillna({name:'0'})



In [0]:
Join_df.createOrReplaceTempView("mytable")

In [0]:
dfdf = spark.table("mytable")
display(dfdf.select("*"))

CountryName,Date,new_cases,COVID_search_index,new_cases_rate,covid_search_rate,Education_search_index,education_rate,Flight_search_index,flight_rate,Export_search_index,export_rate,Immigration_search_index,immigration_rate,Lockdown_search_index,lockdown_rate,Marriage_search_index,marriage_rate,Mask_search_index,mask_rate,Medical_search_index,medicaltreatment_rate,Social_search_index,socialdistance_rate,Travel_search_index,travel_rate,Vaccine_search_index,vaccine_rate,WFH_search_index,wfh_rate
Australia,2020-01-26,12.0,0.0,1,0,65.0,1,100.0,1,63.0,1,68.0,1,1.0,1,94.0,1,29.0,1,67.0,1,0,0,51.0,1,2.0,1,3.0,1
Australia,2020-02-02,3.0,0.0,0.25,0,68.0,1.05,97.0,0.97,69.0,1.10,73.0,1.07,1.0,1.00,87.0,0.93,13.0,0.45,52.0,0.78,0.0,0,52.0,1.02,2.0,1.00,2.0,0.67
Australia,2020-02-09,0.0,0,0,0,69.0,1.01,92.0,0.95,72.0,1.04,64.0,0.88,1.0,1.00,92.0,1.06,10.0,0.77,62.0,1.19,1.0,1,49.0,0.94,2.0,1.00,5.0,2.50
Australia,2020-02-16,0.0,0,0,0,69.0,1.00,83.0,0.90,82.0,1.14,78.0,1.22,0,0,92.0,1.00,8.0,0.80,53.0,0.85,0.0,0,47.0,0.96,2.0,1.00,6.0,1.20
Australia,2020-02-23,10.0,1.0,1,1,69.0,1.00,84.0,1.01,82.0,1.00,77.0,0.99,1.0,1,91.0,0.99,13.0,1.62,68.0,1.28,0,0,72.0,1.53,3.0,1.50,5.0,0.83
Australia,2020-03-01,38.0,2.0,3.80,2.00,64.0,0.93,82.0,0.98,80.0,0.98,73.0,0.95,1.0,1.00,87.0,0.96,14.0,1.08,84.0,1.24,1.0,1,80.0,1.11,5.0,1.67,8.0,1.60
Australia,2020-03-08,187.0,8.0,4.92,4.00,65.0,1.02,69.0,0.84,74.0,0.93,74.0,1.01,9.0,9.00,83.0,0.95,15.0,1.07,68.0,0.81,8.0,8.00,100.0,1.25,6.0,1.20,24.0,3.00
Australia,2020-03-15,821.0,25.0,4.39,3.12,79.0,1.22,94.0,1.36,79.0,1.07,100.0,1.35,37.0,4.11,72.0,0.87,20.0,1.33,88.0,1.29,83.0,10.38,95.0,0.95,8.0,1.33,100.0,4.17
Australia,2020-03-22,2569.0,39.0,3.13,1.56,100.0,1.27,81.0,0.86,71.0,0.90,68.0,0.68,77.0,2.08,68.0,0.94,27.0,1.35,78.0,0.89,100.0,1.20,49.0,0.52,7.0,0.88,100.0,1.00
Australia,2020-03-29,1910.0,33.0,0.74,0.85,82.0,0.82,44.0,0.54,82.0,1.15,61.0,0.90,30.0,0.39,73.0,1.07,25.0,0.93,57.0,0.73,53.0,0.53,29.0,0.59,8.0,1.14,92.0,0.92
